In [ ]:
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict
import networkx as nx
from networkx.algorithms import bipartite

**PUNKT 1**

`Ranking        Wyraz(str).     F[procent] - czestotliwosc.      r x f(słupkami poziomymi)`

In [ ]:
bledne_linie = 0 
raw_text = []
with open("Robinson.txt", "r") as file:
  for line in file:
      if "CHAPTER" in line or len(line)<2:
        bledne_linie+=1
        if "CHAPTER IV" in line: 
          break
      else: raw_text.append(line)
# akapit 1
raw_text[0]

Zrobienie listy slow wszsytkich


In [ ]:
# cleaning and save to list
same_słowa_lista = []
for i in raw_text:
  words = i.split(' ')
  for j in range(len(words)):
    if ',' in words[j]: words[j] = words[j].replace(',','')
    if ';' in words[j]: words[j]= words[j].replace(';','')
    if '(' in words[j]: words[j]= words[j].replace('(','')
    if ')' in words[j]: words[j]= words[j].replace(')','')
    if '»' in words[j]: words[j]= words[j].replace('»','')
    if '«' in words[j]: words[j]= words[j].replace('«','')
    if '.' in words[j]: words[j]= words[j].replace('.','')
    if '\n' in words[j]: words[j]= words[j].replace('\n','')
    if ':' in words[j]: words[j]= words[j].replace(':','')
  same_słowa_lista.append(words)

same_slowa = []

for i in same_słowa_lista:
  for j in i:
    same_slowa.append(j)

print(len(same_slowa))


In [ ]:
counted: List[Tuple[str, int]] = [(unique_word, same_slowa.count(unique_word)) for unique_word in filter(lambda word: word, set(same_slowa))]

In [ ]:
counted.sort(key=lambda x: -x[1])

In [ ]:
print("rank|word|ratio|chart")
ytics: List[str] = []
ratios: List[float] = []
for idx, (word, occurences) in enumerate(counted, 1):
  ratio: float = occurences/len(raw_text)*100
  ytics.append(f'{idx} | {word} | {ratio} | ')
  ratios.append(ratio*idx)

plt.figure(figsize=(30, 650))
plt.barh(ytics, ratios)
# plt.xlim([0,5])
plt.grid()
plt.xlabel("r x f")
plt.show()


**PUNKT 2**

In [ ]:
unique_words: List[str] = [word for word, _ in counted]


# dla kazdego wyrazu z listy unique_words musimy znalezc ten wyraz w licie raw_text i do niego dopisac sasiadow
nested_structure: Dict[str, Dict[str, int]] = {}
plain_structure: Dict[str, int] = {unique_word: 0 for unique_word in unique_words}
for idx, word in enumerate(same_slowa[:-2]):
    for neigh_idx, neigh in enumerate(same_slowa[idx:idx+3]):
        a, b = (word, neigh) if word<neigh else (neigh, word)
        if not a in nested_structure:
            nested_structure[a] = {}
        if not b in nested_structure[a]:
            nested_structure[a][b] = 0
        nested_structure[a][b] += 1
        plain_structure[a]+=1
        plain_structure[b]+=1
        

In [ ]:
N: int = 20
links_threshold: int = 10

G = nx.Graph()
G.add_nodes_from([node for node, links_count in plain_structure.items() if links_count > links_threshold], bipartite=0)
G.add_nodes_from([node+' ' for node, links_count in plain_structure.items() if links_count > links_threshold], bipartite=1)
G.add_edges_from((unique_word, neighbour+" ") for unique_word, neighbours in nested_structure.items() for neighbour, count in neighbours.items() if count > N)


is_bipartite: bool = bipartite.is_bipartite(G)

In [ ]:
plt.hist([count for neighbours in nested_structure.values() for count in neighbours.values()], bins=500)
plt.xlim([0, 20])

In [ ]:
print(max(plain_structure, key = lambda key: plain_structure[key]))

In [ ]:
plt.figure(figsize=(200,200)) #???
nx.draw_networkx(G, pos = nx.drawing.layout.bipartite_layout(G, [word for word in unique_words]), width = 2, font_size=30)

zadanie 3

In [ ]:
pary_slow = []
for i in same_słowa_lista :
  for j in range(len(i)-1) :
    pary_slow.append([i[j], i[j+1]])

print("Ilość par słow wg jednego akapitu: ", len(pary_slow))

# unikalna lista par
unique_pary_slow = []
for i in pary_slow:
    if i not in unique_pary_slow:
        unique_pary_slow.append(i)
print("Ilość unikalnych par słow: " , len(unique_pary_slow))


counter = 0
for i in unique_pary_slow:
  for j in pary_slow:
    if(i == j): counter += 1
  i.append(counter)
  counter = 0

# posortowanie po ilosci wystapien
unique_pary_slow.sort(key = lambda i: -i[2])

# wyswietlenie wyniku
for i in unique_pary_slow:
  print(i[0],"   ", i[1], "  ilośc wystapień: ", i[2])